In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sqlalchemy import create_engine

c:\Users\MishaV\Desktop\karpov\ML\L22\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

In [ ]:
posts = pd.read_csv("/kaggle/input/posts-table/posts.csv", index_col="post_id")
print(posts.shape)
posts.head()

(7023, 2)


,text,topic
post_id,,
1,UK economy facing major risks\n\nThe UK manufa...,business
2,Aids and climate top Davos agenda\n\nClimate c...,business
3,Asian quake hits European shares\n\nShares in ...,business
4,India power shares jump on debut\n\nShares in ...,business
5,Lacroix label bought by US firm\n\nLuxury good...,business


In [ ]:
posts["length"] = posts["text"].str.len()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Проверяем доступность GPU и используем его, если доступно
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка модели и токенизатора
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)  # Перемещаем модель на GPU

# Функция для получения эмбеддингов
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Перемещаем входные данные на GPU
    with torch.no_grad():
        outputs = model(**inputs)
    # Берем эмбеддинг [CLS]-токена (первый токен) и перемещаем обратно на CPU для преобразования в numpy
    cls_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return cls_embedding.flatten()

# Получаем эмбеддинги и добавляем их в DataFrame
embeddings = posts["text"].apply(get_bert_embedding)
for i in range(10):  # берем первые 10 значений из 768-мерного вектора
    posts[f"bert_embedding_{i}"] = embeddings.apply(lambda x: x[i])

print(posts.head())

In [ ]:
posts.to_csv(f"table_posts.csv", index=True)

In [ ]:
posts.reset_index(inplace=True)

In [ ]:
posts.to_sql("mihail_valiev_gjj7899_posts", engine, if_exists="replace", index=False)